In [68]:
import pandas as pd

######## Data intake
cities = pd.read_csv("datasets/cities.csv")
stations = pd.read_csv("datasets/stations.csv")
tracks = pd.read_csv("datasets/tracks.csv")
lines = pd.read_csv("datasets/lines.csv")
track_lines = pd.read_csv("datasets/track_lines.csv")
station_lines = pd.read_csv("datasets/station_lines.csv")
systems = pd.read_csv("datasets/systems.csv")
modes = pd.read_csv("datasets/modes.csv")

In [69]:
# Select and rename columns needed for merging in one go
cities_info = cities[['id', 'country', 'name']].rename(columns={'id': 'city_id', 'name': 'city'})
lines_info = lines[['id', 'name', 'color', 'system_id','transport_mode_id']].rename(columns={'id': 'line_id', 'name': 'line_name', 'color': 'line_color'})
systems_info = systems[['id', 'name']].rename(columns={'id': 'system_id', 'name': 'system_name'})
track_lines_info = track_lines[['section_id', 'line_id','fromyear','toyear']]
station_lines_info = station_lines[['station_id', 'line_id','fromyear','toyear']]

In [70]:
# get mode for each line from modes and then merge 
modes_info = modes[['id', 'name']].rename(columns={'id': 'transport_mode_id', 'name': 'transport_mode_name'})
modes_info['transport_mode_name'] = modes_info['transport_mode_name'].str.replace('_', ' ')
modes_info['transport_mode_name'] = modes_info['transport_mode_name'].str.title()

lines_info = lines_info.merge(modes_info, on='transport_mode_id', how='left')

In [71]:
# --- Process STATIONS ---
stations = stations.rename(columns={'id': 'station_id', 'name': 'station_name'})

# Merge
stations = (stations
            .merge(cities_info, on='city_id', how='left')
            .merge(station_lines_info, on='station_id', how='left')
            .merge(lines_info, on='line_id', how='left')
            .merge(systems_info, on='system_id', how='left')
           )

# Extract coordinates
stations['longitude'] = stations['geometry'].str.extract(r'POINT\(([^ ]+) [^)]+\)')[0].astype(float)
stations['latitude'] = stations['geometry'].str.extract(r'POINT\([^ ]+ ([^)]+)\)')[0].astype(float)

station_cols = ['station_id', 'station_name', 'geometry', 'longitude', 'latitude',
                'opening', 'closure', 'city_id', 'city', 'country',
                'line_id', 'line_name', 'line_color', 'system_id', 'system_name',
                'transport_mode_id', 'transport_mode_name','fromyear','toyear']
stations = stations[station_cols]

In [72]:
# --- Process TRACKS ---
tracks = tracks.rename(columns={'id': 'section_id'})

# Merge
tracks = (tracks
          .merge(cities_info, on='city_id', how='left')
          .merge(track_lines_info, on='section_id', how='left')
          .merge(lines_info, on='line_id', how='left')
          .merge(systems_info, on='system_id', how='left')
         )

track_cols = ['section_id', 'geometry', 'opening', 'closure', 'length',
              'line_id', 'line_name', 'line_color', 'system_id', 'system_name',
              'city_id', 'city', 'country', 'transport_mode_id', 'transport_mode_name','fromyear','toyear']
tracks = tracks[track_cols]

In [73]:

# --- Handle name missing data ---

## Fill in NA 
stations['station_name'] = stations['station_name'].fillna('')
stations['line_name'] = stations['line_name'].fillna('')
stations['line_color'] = stations['line_color'].fillna('#000000')
tracks['line_name'] = tracks['line_name'].fillna('')
tracks['line_color'] = tracks['line_color'].fillna('#000000')
stations['line_id'] = stations['line_id'].fillna(0)
tracks['line_id'] = tracks['line_id'].fillna(0)

# --- Handle opening and closure date missing data ---

# Handle Closure Dates, Fill NaN with 999999, "still open".
stations['closure'] = stations['closure'].fillna(999999)
tracks['closure'] = tracks['closure'].fillna(999999)

# Handle Opening Dates, impute missing data based on minimum known opening year for each city
stations['opening'] = pd.to_numeric(stations['opening'], errors='coerce')
tracks['opening'] = pd.to_numeric(tracks['opening'], errors='coerce')
stations.loc[stations['opening'] > 2100, 'opening'] = pd.NA
tracks.loc[tracks['opening'] > 2100, 'opening'] = pd.NA
stations.loc[stations['opening'] == 0, 'opening'] = pd.NA
tracks.loc[tracks['opening'] == 0, 'opening'] = pd.NA
stations['opening'] = stations['opening'].astype('Int64')
tracks['opening'] = tracks['opening'].astype('Int64')

# Impute
stations['min_city_opening'] = stations.groupby('city')['opening'].transform('min')
tracks['min_city_opening'] = tracks.groupby('city')['opening'].transform('min')
stations['opening'] = stations['opening'].fillna(stations['min_city_opening'])
tracks['opening'] = tracks['opening'].fillna(tracks['min_city_opening'])
# Handle cities where all dates were missing (min_city_opening is NaN)
stations['opening'] = stations['opening'].fillna(1900).astype(int) 
tracks['opening'] = tracks['opening'].fillna(1900).astype(int) 
stations = stations.drop(columns=['min_city_opening']) # Drop the temporary column
tracks = tracks.drop(columns=['min_city_opening']) # Drop the temporary column

# Flag rows still missing opening date after imputation
stations['was_missing_opening'] = stations['opening'].isna() 
tracks['was_missing_opening'] = tracks['opening'].isna() 

In [74]:
# ... (wonkiness calculation) ...
# Calculate proportion of missing/imputed opening dates per city
pivot_st = pd.pivot_table(stations, values='station_id', index='city_id', columns='was_missing_opening', aggfunc='count', fill_value=0)
pivot_tr = pd.pivot_table(tracks, values='section_id', index='city_id', columns='was_missing_opening', aggfunc='count', fill_value=0)

pivot_st = pivot_st.rename(columns={True: 'missing', False: 'valid'})
pivot_tr = pivot_tr.rename(columns={True: 'missing', False: 'valid'})
if 'missing' not in pivot_st.columns: pivot_st['missing'] = 0
if 'valid' not in pivot_st.columns: pivot_st['valid'] = 0
if 'missing' not in pivot_tr.columns: pivot_tr['missing'] = 0
if 'valid' not in pivot_tr.columns: pivot_tr['valid'] = 0

# Calculate the "wonkiness" score 9missing dates) then merge for filtering
pivot_st['wonkiness_st'] = pivot_st['missing'] / (pivot_st['valid'] + pivot_st['missing'])
pivot_tr['wonkiness_tr'] = pivot_tr['missing'] / (pivot_tr['valid'] + pivot_tr['missing'])
wonk_table = pd.merge(pivot_st[['valid', 'missing', 'wonkiness_st']],
                      pivot_tr[['valid', 'missing', 'wonkiness_tr']],
                      on='city_id', suffixes=('_st', '_tr'))

wonk_table = wonk_table.merge(cities_info[['city_id','city', 'country']], on='city_id', how='left')
wonk_table['total_stations'] = wonk_table['valid_st'] + wonk_table['missing_st']
wonk_table['total_tracks'] = wonk_table['valid_tr'] + wonk_table['missing_tr']

# wonk_table['avg_wonkiness'] = (wonk_table['wonkiness_st'] + wonk_table['wonkiness_tr']) / 2 
wonk_table['avg_wonkiness'] = wonk_table['wonkiness_tr'] # wonk based on tracks only

In [75]:
# --- Extract LineString coordinates from WKT strings ---
import re
def parse_linestring_coords(wkt_string, swap_order=False, precision=6):
    if not isinstance(wkt_string, str):
        return None 

    # Regular expression to find all coordinate pairs (lon lat)
    coord_pattern = re.compile(r"(-?\d+\.?\d*)\s+(-?\d+\.?\d*)")
    matches = coord_pattern.findall(wkt_string)
    if not matches:
        return None

    coord_list = []
    for lon_str, lat_str in matches:
        try:
            lon = round(float(lon_str), precision)
            lat = round(float(lat_str), precision)
            if swap_order:
                coord_list.append([lat, lon]) # Append as [lat, lon]
            else:
                coord_list.append([lon, lat]) # Append as [lon, lat]
        except ValueError:
            print(f"WARN: Could not parse coordinate pair: ('{lon_str}', '{lat_str}') in '{wkt_string}'")
            continue 

    return coord_list if coord_list else None

tracks['linestring_lonlat'] = tracks['geometry'].apply(
    lambda x: parse_linestring_coords(x, swap_order=False)
) # to be used for export
tracks['linestring_latlon'] = tracks['geometry'].apply(
    lambda x: parse_linestring_coords(x, swap_order=True)
) # to be used for map plotting dl

In [76]:
# export
stations.to_csv(
  "stations_cleaned.csv", 
  index=False, 
  encoding='utf-8', 
  )
tracks.to_csv(
  "tracks_cleaned.csv", 
  index=False, 
  encoding='utf-8', 
  )
wonk_table.to_csv(
  "city_wonkiness.csv", 
  index=True,
  encoding='utf-8', 
  )